<a href="https://colab.research.google.com/github/SaahithShetty/tensorflow/blob/main/01_neural_network_regression_with_tf_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Imoprt required libraries
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from sklearn.model_selection import train_test_split

In [4]:
insurance = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/refs/heads/master/insurance.csv")
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
#One hot encode the data
insurance_one_hot = pd.get_dummies(insurance,dtype=int)
insurance_one_hot.head()

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0


In [6]:
X = insurance_one_hot.drop("charges",axis=1)
y = insurance_one_hot["charges"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.head()

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
560,46,19.95,2,1,0,1,0,0,1,0,0
1285,47,24.32,0,1,0,1,0,1,0,0,0
1142,52,24.86,0,1,0,1,0,0,0,1,0
969,39,34.32,5,1,0,1,0,0,0,1,0
486,54,21.47,3,1,0,1,0,0,1,0,0


In [8]:
# BUild a model
tf.random.set_seed(42)
insurance_model = tf.keras.Sequential([
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1)
])

insurance_model.compile(
    loss = tf.keras.losses.mae,
    optimizer = tf.keras.optimizers.SGD(),
    metrics = ["mae"]
)

insurance_model.fit(X_train,y_train,epochs=100)

Epoch 1/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 10340.5293 - mae: 10340.5293   
Epoch 2/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7690.9053 - mae: 7690.9053 
Epoch 3/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7774.9717 - mae: 7774.9717 
Epoch 4/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7724.9360 - mae: 7724.9360 
Epoch 5/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7803.4092 - mae: 7803.4092 
Epoch 6/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7299.1011 - mae: 7299.1011
Epoch 7/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7485.6562 - mae: 7485.6562 
Epoch 8/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7334.0571 - mae: 7334.0571 
Epoch 9/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7360.3799 - mae: 7360.3799 
Epoch 10/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7367.7300 - mae: 7367.7300 
Epoch 11/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7416.9302 - mae: 7416.9302
Epoch 12/100
34/34 ━━━━━━━━━━━━━━━━━━━━

In [9]:
# Check the results of the insurance model on the test data
insurance_model.evaluate(X_test,y_test)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8450.9648 - mae: 8450.9648  


[8444.623046875, 8444.623046875]

In [13]:
# Now let recreate this with Pytroch
# Convert thwe data intro tensors
X_train_tensor = torch.from_numpy(X_train.to_numpy()).type(torch.float32)
X_test_tensor = torch.from_numpy(X_test.to_numpy()).type(torch.float32)
y_train_tensor = torch.from_numpy(y_train.to_numpy()).type(torch.float32)
y_test_tensor = torch.from_numpy(y_test.to_numpy()).type(torch.float32)
X_train_tensor.shape

torch.Size([1070, 11])

In [30]:
class InsuranceModelPytorch(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(in_features=11,out_features=10)
    self.layer_2 = nn.Linear(in_features=10,out_features=1)
  def forward(self,x):
    return self.layer_2(self.layer_1(x))

In [31]:
insurance_model_pytorch = InsuranceModelPytorch()
insurance_model_pytorch

InsuranceModelPytorch(
  (layer_1): Linear(in_features=11, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=1, bias=True)
)

In [32]:
#Lets train the model
EPOCHS = 100
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=insurance_model_pytorch.parameters(),lr=0.01)

In [33]:
torch.manual_seed(42)
for epoch in range(EPOCHS):
  insurance_model_pytorch.train()
  y_pred = insurance_model_pytorch(X_train_tensor)
  loss = loss_fn(y_pred,y_train_tensor)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  insurance_model_pytorch.eval()
  with torch.inference_mode():
    test_pred = insurance_model_pytorch(X_test_tensor)
    test_loss = loss_fn(test_pred,y_test_tensor)
  print(f"Epoch: {epoch} | Loss: {loss} | Test Loss: {test_loss}")

Epoch: 0 | Loss: 13350.953125 | Test Loss: 12958.7958984375
Epoch: 1 | Loss: 13336.623046875 | Test Loss: 12942.203125
Epoch: 2 | Loss: 13319.9619140625 | Test Loss: 12908.8291015625
Epoch: 3 | Loss: 13286.4541015625 | Test Loss: 12834.8916015625
Epoch: 4 | Loss: 13212.1865234375 | Test Loss: 12669.1474609375
Epoch: 5 | Loss: 13045.6474609375 | Test Loss: 12297.0595703125
Epoch: 6 | Loss: 12671.67578125 | Test Loss: 11479.45703125
Epoch: 7 | Loss: 11849.720703125 | Test Loss: 10151.041015625
Epoch: 8 | Loss: 10467.529296875 | Test Loss: 9061.6201171875
Epoch: 9 | Loss: 9133.705078125 | Test Loss: 8772.75390625
Epoch: 10 | Loss: 8586.8330078125 | Test Loss: 8801.501953125
Epoch: 11 | Loss: 8535.2333984375 | Test Loss: 8805.2060546875
Epoch: 12 | Loss: 8531.435546875 | Test Loss: 8803.0830078125
Epoch: 13 | Loss: 8528.0751953125 | Test Loss: 8800.9267578125
Epoch: 14 | Loss: 8524.8154296875 | Test Loss: 8798.732421875
Epoch: 15 | Loss: 8521.6591796875 | Test Loss: 8796.689453125
Epoch: 1

Lets do 2 experiments to improve our model
1. Add an extra layer with more hidden units
2. Train for longer

In [41]:
tf.random.set_seed(42)

#Create the model
insurance_model_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1)
])

#Compile the model
insurance_model_2.compile(
    loss = tf.keras.losses.mae,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["mae"]
)

insurance_model_2.fit(X_train,y_train,epochs=100)

Epoch 1/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 13293.9150 - mae: 13293.9150
Epoch 2/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 13195.0410 - mae: 13195.0410
Epoch 3/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12998.8965 - mae: 12998.8965
Epoch 4/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12569.6328 - mae: 12569.6328
Epoch 5/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11735.7051 - mae: 11735.7051
Epoch 6/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10410.9170 - mae: 10410.9170
Epoch 7/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8888.9189 - mae: 8888.9189
Epoch 8/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7735.4600 - mae: 7735.4600
Epoch 9/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7405.2358 - mae: 7405.2358
Epoch 10/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7362.8770 - mae: 7362.8770
Epoch 11/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7341.8389 - mae: 7341.8389
Epoch 12/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 

In [42]:
#Lets evaluate the model
insurance_model_2.evaluate(X_test,y_test)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5156.1777 - mae: 5156.1777  


[4999.06787109375, 4999.06787109375]

In [44]:
# Lets recreate the model with pytorch

class TwoLayerInsuranceModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(in_features=11,out_features=100)
    self.layer_2 = nn.Linear(in_features=100,out_features=10)
    self.layer_3 = nn.Linear(in_features=10,out_features=1)

  def forward(self,x):
    return self.layer_3(self.layer_2(self.layer_1(x)))

In [50]:
pytorch_insurance_model_2 = TwoLayerInsuranceModel()
pytorch_insurance_model_2

TwoLayerInsuranceModel(
  (layer_1): Linear(in_features=11, out_features=100, bias=True)
  (layer_2): Linear(in_features=100, out_features=10, bias=True)
  (layer_3): Linear(in_features=10, out_features=1, bias=True)
)

In [55]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(params=pytorch_insurance_model_2.parameters(),lr=0.001)

In [56]:
EPOCHS = 100

In [58]:
for epoch in range(EPOCHS):
  pytorch_insurance_model_2.train()
  y_pred = pytorch_insurance_model_2(X_train_tensor)
  loss = loss_fn(y_pred,y_train_tensor)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  pytorch_insurance_model_2.eval()
  with torch.inference_mode():
    test_pred = pytorch_insurance_model_2(X_test_tensor)
    test_loss = loss_fn(test_pred,y_test_tensor)
  print(f"Epoch: {epoch+1} | Loss: {loss} | Test Loss: {test_loss}")

Epoch: 1 | Loss: 8428.2744140625 | Test Loss: 8741.35546875
Epoch: 2 | Loss: 8428.00390625 | Test Loss: 8741.234375
Epoch: 3 | Loss: 8427.7353515625 | Test Loss: 8741.1083984375
Epoch: 4 | Loss: 8427.466796875 | Test Loss: 8740.9736328125
Epoch: 5 | Loss: 8427.201171875 | Test Loss: 8740.8310546875
Epoch: 6 | Loss: 8426.9375 | Test Loss: 8740.6806640625
Epoch: 7 | Loss: 8426.6748046875 | Test Loss: 8740.521484375
Epoch: 8 | Loss: 8426.412109375 | Test Loss: 8740.359375
Epoch: 9 | Loss: 8426.1533203125 | Test Loss: 8740.1962890625
Epoch: 10 | Loss: 8425.89453125 | Test Loss: 8740.03125
Epoch: 11 | Loss: 8425.6376953125 | Test Loss: 8739.87109375
Epoch: 12 | Loss: 8425.3828125 | Test Loss: 8739.7216796875
Epoch: 13 | Loss: 8425.1279296875 | Test Loss: 8739.58203125
Epoch: 14 | Loss: 8424.875 | Test Loss: 8739.4453125
Epoch: 15 | Loss: 8424.625 | Test Loss: 8739.3095703125
Epoch: 16 | Loss: 8424.3759765625 | Test Loss: 8739.1748046875
Epoch: 17 | Loss: 8424.126953125 | Test Loss: 8739.038